In [6]:
import requests
import json
import torch
import glob
import os
import numpy as np
import shutil
import argparse
import logging
import pandas as pd
from PIL import Image, UnidentifiedImageError
from torchvision import transforms
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
import warnings
import time
from tqdm import tqdm
from collections import Counter
from torch.cuda import OutOfMemoryError

In [2]:
# transform = transforms.Compose([
#     transforms.Resize((120, 120))])

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 65
CLS_DICT = {'0': 'Bird',
 '1': 'Ground-Animal',
 '2': 'Curb',
 '3': 'Fence',
 '4': 'Guard-Rail',
 '5': 'Barrier',
 '6': 'Wall',
 '7': 'Bike-Lane',
 '8': 'Crosswalk---Plain',
 '9': 'Curb-Cut',
 '10': 'Parking',
 '11': 'Pedestrian-Area',
 '12': 'Rail-Track',
 '13': 'Road',
 '14': 'Service-Lane',
 '15': 'Sidewalk',
 '16': 'Bridge',
 '17': 'Building',
 '18': 'Tunnel',
 '19': 'Person',
 '20': 'Bicyclist',
 '21': 'Motorcyclist',
 '22': 'Other-Rider',
 '23': 'Lane-Marking---Crosswalk',
 '24': 'Lane-Marking---General',
 '25': 'Mountain',
 '26': 'Sand',
 '27': 'Sky',
 '28': 'Snow',
 '29': 'Terrain',
 '30': 'Vegetation',
 '31': 'Water',
 '32': 'Banner',
 '33': 'Bench',
 '34': 'Bike-Rack',
 '35': 'Billboard',
 '36': 'Catch-Basin',
 '37': 'CCTV-Camera',
 '38': 'Fire-Hydrant',
 '39': 'Junction-Box',
 '40': 'Mailbox',
 '41': 'Manhole',
 '42': 'Phone-Booth',
 '43': 'Pothole',
 '44': 'Street-Light',
 '45': 'Pole',
 '46': 'Traffic-Sign-Frame',
 '47': 'Utility-Pole',
 '48': 'Traffic-Light',
 '49': 'Traffic-Sign-(Back)',
 '50': 'Traffic-Sign-(Front)',
 '51': 'Trash-Can',
 '52': 'Bicycle',
 '53': 'Boat',
 '54': 'Bus',
 '55': 'Car',
 '56': 'Caravan',
 '57': 'Motorcycle',
 '58': 'On-Rails',
 '59': 'Other-Vehicle',
 '60': 'Trailer',
 '61': 'Truck',
 '62': 'Wheeled-Slow',
 '63': 'Car-Mount',
 '64': 'Ego-Vehicle'}

# Get helper function
def addIndice(output_max):
    set_of_pixels = torch.unique(output_max, return_counts=True)
    set_dictionary = {}
    for i in range(NUM_CLASSES):
            set_dictionary[str(i)] = 0
    for pixel,count in zip(set_of_pixels[0], set_of_pixels[1]):
        set_dictionary[str(pixel.item())] = count.item()
    set_dictionary['Total'] = int(np.sum(list(set_dictionary.values())))
    return set_dictionary

def addInstance(output_max):
    list_unique, list_counts = torch.unique(out[0]['segmentation'].int(), return_counts=True)

    if -1 in list_unique:
        list_unique = list_unique[1:]
        list_counts = list_counts[1:]

    total = torch.sum(list_counts).item()

    matching_dict = {}
    for i, k in zip(range(len(out[0]['segments_info'])), out[0]['segments_info']):
        matching_dict[i] = int(k['label_id'])

    set_dictionary = {}
    for i in range(NUM_CLASSES):
                set_dictionary[str(i)] = 0

    for i, k in zip(list_unique, list_counts):
        set_dictionary[str(matching_dict[i.item()])] += k.item()
        
    set_dictionary['Total'] = total

    return set_dictionary

def addInstanceCounts(output_max):

    instance_dictionary = {}
    
    instance_dictionary = {}
    for i in range(NUM_CLASSES):
                instance_dictionary[str(i)] = 0
    
    # for each segment, draw its legend
    for segment in out[0]['segments_info']:
        segment_id = segment['id']
        segment_label_id = str(segment['label_id'])
        instance_dictionary[segment_label_id] += 1

    return instance_dictionary

# Load Mask2Former
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-mapillary-vistas-panoptic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-mapillary-vistas-panoptic")
model = model.to(device)

In [4]:
print(device)

cuda


In [5]:
# Set start state
image_indicators_dict = {}
image_instances_dict = {}

# Create output folder if none exist
if not os.path.exists(f'./outputs'):
    os.makedirs(f'./outputs')

# Get list of images
image_set = [i for i in os.listdir(os.path.join(os.getcwd(),'/media/ual/UAL-PSSD-1/global_streetscapes_imgs_1/'))]

In [6]:
len(image_set)

1677722

In [7]:
for i, image in enumerate(tqdm(image_set[0:])):
    print(f'Segmenting {image}')
    img = Image.open(os.path.join(os.getcwd(),'/media/ual/UAL-PSSD-1/global_streetscapes_imgs_1/', f'{image}'))
    inputs = processor(images=img, return_tensors="pt", height = 120, width=120)
    with torch.no_grad():
        pixel_values = inputs['pixel_values'].to(device)
        pixel_mask = inputs['pixel_mask'].to(device)
        outputs = model(pixel_values = pixel_values, pixel_mask = pixel_mask)
        outputs['class_queries_logits'] = outputs['class_queries_logits'].to('cpu')
        outputs['masks_queries_logits'] = outputs['masks_queries_logits'].to('cpu')
        outputs['encoder_last_hidden_state'] = outputs['encoder_last_hidden_state'].to('cpu')
        outputs['pixel_decoder_last_hidden_state'] = outputs['pixel_decoder_last_hidden_state'].to('cpu')
        outputs['transformer_decoder_last_hidden_state'] = outputs['transformer_decoder_last_hidden_state'].to('cpu')
    out = processor.post_process_instance_segmentation(outputs, target_sizes=[img.size[::-1]], threshold=0.25)
    image_indicators_dict[image] = addInstance(out)
    image_instances_dict[image] = addInstanceCounts(out)

  0%|                                                                                                                                                                          | 0/1677722 [00:00<?, ?it/s]

Segmenting 278f7c8a-3063-4acd-ae93-4c6ff848c8d3.jpeg


/home/ual/.conda/envs/svi_global/lib/python3.9/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
  0%|                                                                                                                                                             | 1/1677722 [00:04<1893:00:39,  4.06s/it]

Segmenting b6a4d5a1-dde2-40c9-9c55-bd6fb3431bef.jpeg


  0%|                                                                                                                                                             | 2/1677722 [00:07<1620:57:37,  3.48s/it]

...

Segmenting 10b32d5f-84d7-486e-9c6b-7fe9b64385b0.jpeg


  0%|                                                                                                                                                            | 12/1677722 [00:37<1388:56:23,  2.98s/it]

Segmenting 6f032d01-f078-4c4c-b2dc-92b8ed5da2f1.jpeg


  0%|                                                                                                                                                            | 12/1677722 [00:38<1477:26:33,  3.17s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 6.18 GiB (GPU 0; 7.92 GiB total capacity; 1.17 GiB already allocated; 5.75 GiB free; 1.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
img.size

In [ ]:
print(f'Segmenting {image}')
img = Image.open(os.path.join(os.getcwd(),'/media/ual/UAL-PSSD-1/global_streetscapes_imgs_1/', '6f032d01-f078-4c4c-b2dc-92b8ed5da2f1.jpeg'))
inputs = processor(images=img, return_tensors="pt")
with torch.no_grad():
    pixel_values = inputs['pixel_values'].to(device)
    pixel_mask = inputs['pixel_mask'].to(device)
    outputs = model(pixel_values = pixel_values, pixel_mask = pixel_mask)
    outputs['class_queries_logits'] = outputs['class_queries_logits'].to('cpu')
    outputs['masks_queries_logits'] = outputs['masks_queries_logits'].to('cpu')
    outputs['encoder_last_hidden_state'] = outputs['encoder_last_hidden_state'].to('cpu')
    outputs['pixel_decoder_last_hidden_state'] = outputs['pixel_decoder_last_hidden_state'].to('cpu')
    outputs['transformer_decoder_last_hidden_state'] = outputs['transformer_decoder_last_hidden_state'].to('cpu')
out = processor.post_process_instance_segmentation(outputs, target_sizes=[img.size[::-1]], threshold=0.25)
image_indicators_dict[image] = addInstance(out)
image_instances_dict[image] = addInstanceCounts(out)

In [ ]:
outputs[0] = outputs[0].to('cpu')

In [ ]:
transform(img)

In [ ]:
img

In [ ]:
df = pd.DataFrame.from_dict(image_instances_dict, orient='index')

In [ ]:
image_instances_dict['139784358078458.jpg']

In [ ]:
img = Image.open(os.path.join(os.getcwd(),'test/', '139784358078458.jpg'))

In [ ]:
img

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import cm

In [ ]:
plt.imshow(out[0]['segmentation'])

### Merge CSV

In [ ]:
from glob import glob
import pandas as pd

In [ ]:
paths[0]

In [ ]:
out2 = pd.read_csv(paths[1])

In [ ]:
total = pd.DataFrame()

In [ ]:
total = pd.concat([total, out2], axis=0)

In [ ]:
def merge_csv(filepath):

    extension = filepath + '*.csv'
    paths = glob(extension)
    
    combined = pd.DataFrame()
    for path in paths:
        temp = pd.read_csv(path)
        combined = pd.concat([combined, temp], axis=0)
    
    return combined

In [ ]:
all_df = merge_csv('./outputs/')

In [ ]:
len(all_df)

In [ ]:
all_df.to_csv('./final/ssd1_1676060.csv')

In [ ]:
# Closing log file

with open('yourlog.log', 'w'):
    pass
